https://github.com/kushalj001/pytorch-question-answering/blob/master/1.%20DrQA.ipynb

In [1]:
import pandas as pd
import numpy as np
import torchtext
import torch
from torch import nn
import json, re, unicodedata, string, typing, time
import torch.nn.functional as F
import spacy
from collections import Counter
import pickle
import Bert_Question_Answering_preprocessing as BQ
from nltk import word_tokenize
nlp = spacy.load('en')
%load_ext autoreload
%autoreload 2

In [2]:
train_data = BQ.load_json(r'C:\Users\abc\jupyter\pytorch\Bert\squad_train.json')
valid_data = BQ.load_json(r'C:\Users\abc\jupyter\pytorch\Bert\squad_dev.json')

train_list = BQ.parse_data(train_data)
valid_list = BQ.parse_data(valid_data)

print('Train list len : ', len(train_list))
print('Valid list lne : ', len(valid_list))

train_df = pd.DataFrame(train_list)
valid_df = pd.DataFrame(valid_list)

Length of data:  442
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  Beyoncé
Length of data:  35
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  Normans
Train list len :  86821
Valid list lne :  20302


In [3]:
def normalize_spaces(text):
    """
    정규화 과정
    """
    text = re.sub(r'\s', ' ',text)
    return text

train_df.context = train_df.context.apply(normalize_spaces)
valid_df.context = valid_df.context.apply(normalize_spaces)
                  

In [4]:
train_df.head()

,id,context,question,label,answer
0,56be85543aeaaa14008c9063,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"[269, 286]",in the late 1990s
1,56be85543aeaaa14008c9065,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,"[207, 226]",singing and dancing
2,56be85543aeaaa14008c9066,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,"[526, 530]",2003
3,56bf6b0f3aeaaa14008c9601,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"[166, 180]","Houston, Texas"
4,56bf6b0f3aeaaa14008c9602,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,"[276, 286]",late 1990s


In [5]:
%time vocab_text = BQ.gather_text_for_vocab([train_df, valid_df])
print("Number of sentences in dataset: ", len(vocab_text))

Wall time: 349 ms
Number of sentences in dataset:  112776


In [6]:
%time word2idx, idx2word, word_vocab = BQ.build_word_vocab(vocab_text)

raw-vocab: 108286
glove-vocab: 108286
vocab-length: 108288
word2idx-length: 108288
Wall time: 23.2 s


In [7]:
%time train_df['context_ids'] = train_df.context.apply(BQ.context_to_ids, word2idx=word2idx)
%time valid_df['context_ids'] = valid_df.context.apply(BQ.context_to_ids, word2idx=word2idx)

%time train_df['question_ids'] = train_df.question.apply(BQ.question_to_ids, word2idx = word2idx)
%time valid_df['question_ids'] = valid_df.question.apply(BQ.question_to_ids, word2idx = word2idx)

Wall time: 1min 8s
Wall time: 17.2 s
Wall time: 6.18 s
Wall time: 1.48 s


In [8]:
train_err = BQ.get_error_indices(train_df, idx2word)
valid_err = BQ.get_error_indices(valid_df, idx2word)

train_df.drop(train_err, inplace=True)
valid_df.drop(valid_err, inplace=True)

Error indices: 986
Error indices: 208


In [10]:
train_label_idx = train_df.apply(BQ.index_answer, axis=1, idx2word=idx2word)
valid_label_idx = valid_df.apply(BQ.index_answer, axis=1, idx2word=idx2word)

train_df['label_idx'] = train_label_idx
valid_df['label_idx'] = valid_label_idx

In [11]:
import pickle
with open('drqastoi.pickle','wb') as handle:
    pickle.dump(word2idx, handle)
    
train_df.to_pickle('drqatrain.pkl')
valid_df.to_pickle('drqavalid.pkl')

In [12]:
train_df = pd.read_pickle('drqatrain.pkl')
valid_df = pd.read_pickle('drqavalid.pkl')

In [13]:
class SquadDataset:
    '''
    -Divides the dataframe in batches.
    -Pads the contexts and questions dynamically for each batch by padding 
     the examples to the maximum-length sequence in that batch.
    -Calculates masks for context and question.
    -Calculates spans for contexts.
    '''
    
    def __init__(self, data, batch_size):
        
        self.batch_size = batch_size
        data = [data[i:i+self.batch_size] for i in range(0, len(data), self.batch_size)]
        self.data = data
    
    def get_span(self, text):
        
        text = nlp(text, disable=['parser','tagger','ner'])
        span = [(w.idx, w.idx+len(w.text)) for w in text]

        return span

    def __len__(self):
        return len(self.data)
    
    def __iter__(self):
        '''
        Creates batches of data and yields them.
        
        Each yield comprises of:
        :padded_context: padded tensor of contexts for each batch 
        :padded_question: padded tensor of questions for each batch 
        :context_mask & question_mask: zero-mask for question and context
        :label: start and end index wrt context_ids
        :context_text,answer_text: used while validation to calculate metrics
        :context_spans: spans of context text
        :ids: question_ids used in evaluation
        '''
        
        for batch in self.data:
                            
            spans = []
            context_text = []
            answer_text = []
            
            max_context_len = max([len(ctx) for ctx in batch.context_ids])
            padded_context = torch.LongTensor(len(batch), max_context_len).fill_(1)
            
            for ctx in batch.context:
                context_text.append(ctx)
                spans.append(self.get_span(ctx))
            
            for ans in batch.answer:
                answer_text.append(ans)
                
            for i, ctx in enumerate(batch.context_ids):
                padded_context[i, :len(ctx)] = torch.LongTensor(ctx)
            
            max_question_len = max([len(ques) for ques in batch.question_ids])
            padded_question = torch.LongTensor(len(batch), max_question_len).fill_(1)
            
            for i, ques in enumerate(batch.question_ids):
                padded_question[i,: len(ques)] = torch.LongTensor(ques)
                
            
            label = torch.LongTensor(list(batch.label_idx))
            context_mask = torch.eq(padded_context, 1)
            question_mask = torch.eq(padded_question, 1)
            
            ids = list(batch.id)  
            
            yield (padded_context, padded_question, context_mask, 
                   question_mask, label, context_text, answer_text, ids)

In [14]:
train_dataset = SquadDataset(train_df, 32)
valid_dataset = SquadDataset(valid_df, 32)

In [15]:
a = next(iter(train_dataset))

In [16]:
a[0].shape, a[1].shape, a[2].shape, a[3].shape, a[4].shape

(torch.Size([32, 215]),
 torch.Size([32, 19]),
 torch.Size([32, 215]),
 torch.Size([32, 19]),
 torch.Size([32, 2]))